In [2]:
using Pkg

Pkg.activate("../Project.toml")

using Knet
using Random
using Debugger


 Activating environment at `~/Desktop/MSc/repos/yolo-julia/Project.toml`


In [222]:
include("../coco2014.jl")
x, y = load_data(
    "/home/bcs/Desktop/MSc/repos/yolo-julia/dataset/images/overfit",
    "../val_labels.json",
    class_file="../class_mappings.json"
);

dtrn = minibatch(x, y, 3; xsize = (416,416,3,:), xtype=Knet.atype(), shuffle=true);
xx, yy = first(dtrn);

In [4]:

include("../utils/parse_config.jl")
include("../models.jl")
Random.seed!(1)

darknet = Darknet("../yolov3.cfg"; verbose=false);
yolo_out = darknet(xx);

In [225]:
include("../utils/utils.jl")

dtrn = minibatch(x, y, 3; xsize = (416,416,3,:), xtype=Knet.atype(), shuffle=true);
xx, yy = first(dtrn);
# tcls, tbox, indices, anch = build_targets(yolo_out, yy, darknet)
tcls, tbox, indices, anch = build_targets(yolo_out, yy, darknet);

LoadError: DimensionMismatch("new dimensions (3, 23) must be consistent with array size 39")

In [248]:
tcls[3]

17-element Array{Int64,1}:
 49
 57
  1
  1
  3
 10
 10
 57
 57
  1
 57
  1
  1
  3
 49
 10
 10

In [207]:
gwh

22×2 Array{Float64,2}:
 3.82099   12.7668
 4.01639   11.2871
 0.841347   3.27922
 3.54392    0.611039
 1.94553    7.04704
 1.83747    7.30306
 1.15599    6.0937
 2.821      3.12597
 9.86233   12.4429
 8.34153    9.04732
 3.82099   12.7668
 4.01639   11.2871
 7.38176    7.40737
 1.94553    7.04704
 1.83747    7.30306
 1.15599    6.0937
 2.821      3.12597
 9.86233   12.4429
 8.34153    9.04732
 3.82099   12.7668
 4.01639   11.2871
 7.38176    7.40737

In [190]:
gij = ceil.(Integer, gxy .- offsets)
gi = gij[:, 1]
gj = gij[:, 2]

22-element Array{Int64,1}:
  7
  6
  3
 11
 10
 10
 10
 12
  7
  7
  7
  6
  8
 10
 10
 10
 12
  7
  7
  7
  6
  8

In [191]:

b = Integer.(t[:, 1])
c = Integer.(t[:, 2])
gxy = t[:, 3:4]
gwh = t[:, 5:6]

22×2 Array{Float64,2}:
 3.82099   12.7668
 4.01639   11.2871
 0.841347   3.27922
 3.54392    0.611039
 1.94553    7.04704
 1.83747    7.30306
 1.15599    6.0937
 2.821      3.12597
 9.86233   12.4429
 8.34153    9.04732
 3.82099   12.7668
 4.01639   11.2871
 7.38176    7.40737
 1.94553    7.04704
 1.83747    7.30306
 1.15599    6.0937
 2.821      3.12597
 9.86233   12.4429
 8.34153    9.04732
 3.82099   12.7668
 4.01639   11.2871
 7.38176    7.40737

In [195]:
size(yolo_out[1])

(85, 13, 13, 3, 3)

In [103]:
j

3×13 BitArray{2}:
 0  0  1  1  0  1  1  1  0  1  1  1  0
 1  1  1  1  1  0  0  1  0  1  1  1  0
 1  1  1  1  1  0  0  0  0  0  0  0  0

In [147]:
j'

13×3 LinearAlgebra.Adjoint{Bool,BitArray{2}}:
 0  1  1
 0  1  1
 1  1  1
 1  1  1
 0  1  1
 1  0  0
 1  0  0
 1  1  0
 0  0  0
 1  1  0
 1  1  0
 1  1  0
 0  0  0

In [141]:
tt[jj, :]

22×6 Array{Float64,2}:
 1.0  21.0   5.2451     6.11928  8.34153    9.04732
 1.0  21.0  11.0895     6.41263  3.82099   12.7668
 2.0   1.0   8.8661     7.4662   7.38176    7.40737
 2.0  36.0   0.420667   2.62042  0.841347   3.27922
 2.0  35.0   1.77197   10.2312   3.54392    0.611039
 3.0  73.0   5.39256    9.47648  1.94553    7.04704
 3.0  75.0   9.54261    4.01509  0.472056   0.97838
 3.0   1.0   7.56092    9.29958  1.83747    7.30306
 3.0   1.0   9.50228    9.95315  1.15599    6.0937
 3.0  70.0   1.90308   11.2617   2.821      3.12597
 1.0  21.0   7.01665    6.77858  9.86233   12.4429
 1.0  21.0   5.2451     6.11928  8.34153    9.04732
 1.0  21.0  11.0895     6.41263  3.82099   12.7668
 2.0  36.0   0.420667   2.62042  0.841347   3.27922
 3.0  75.0   9.54261    4.01509  0.472056   0.97838
 3.0   1.0   7.56092    9.29958  1.83747    7.30306
 1.0  21.0   5.2451     6.11928  8.34153    9.04732
 1.0  21.0  11.0895     6.41263  3.82099   12.7668
 2.0   1.0   8.8661     7.4662   7.38176    7

In [140]:
tt = repeat(t, 3)

39×6 Array{Float64,2}:
 1.0  21.0   7.01665    6.77858  9.86233   12.4429
 1.0  21.0   5.2451     6.11928  8.34153    9.04732
 1.0  21.0  11.0895     6.41263  3.82099   12.7668
 2.0   1.0   2.0082     5.64357  4.01639   11.2871
 2.0   1.0   8.8661     7.4662   7.38176    7.40737
 2.0  36.0   0.420667   2.62042  0.841347   3.27922
 2.0  35.0   1.77197   10.2312   3.54392    0.611039
 3.0  73.0   5.39256    9.47648  1.94553    7.04704
 3.0  75.0   9.54261    4.01509  0.472056   0.97838
 3.0   1.0   7.56092    9.29958  1.83747    7.30306
 3.0   1.0   9.50228    9.95315  1.15599    6.0937
 3.0  70.0   1.90308   11.2617   2.821      3.12597
 3.0  27.0   9.70897    9.99132  0.521222   1.56398
 ⋮                                          ⋮
 1.0  21.0   5.2451     6.11928  8.34153    9.04732
 1.0  21.0  11.0895     6.41263  3.82099   12.7668
 2.0   1.0   2.0082     5.64357  4.01639   11.2871
 2.0   1.0   8.8661     7.4662   7.38176    7.40737
 2.0  36.0   0.420667   2.62042  0.841347   3.27922


In [117]:
tt[j]

LoadError: BoundsError: attempt to access 13×6×3 Array{Float64,3} at index [Bool[0 0 1 1 0 1 1 1 0 1 1 1 0; 1 1 1 1 1 0 0 1 0 1 1 1 0; 1 1 1 1 1 0 0 0 0 0 0 0 0]]

In [196]:
at'[jj]

22-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 2
 2
 2
 2
 2
 2
 3
 3
 3
 3
 3
 3

In [40]:
targets_reshaped .* gain

13×6 Array{Float64,2}:
 1.0  21.0   7.01665    6.77858  9.86233   12.4429
 1.0  21.0   5.2451     6.11928  8.34153    9.04732
 1.0  21.0  11.0895     6.41263  3.82099   12.7668
 2.0   1.0   2.0082     5.64357  4.01639   11.2871
 2.0   1.0   8.8661     7.4662   7.38176    7.40737
 2.0  36.0   0.420667   2.62042  0.841347   3.27922
 2.0  35.0   1.77197   10.2312   3.54392    0.611039
 3.0  73.0   5.39256    9.47648  1.94553    7.04704
 3.0  75.0   9.54261    4.01509  0.472056   0.97838
 3.0   1.0   7.56092    9.29958  1.83747    7.30306
 3.0   1.0   9.50228    9.95315  1.15599    6.0937
 3.0  70.0   1.90308   11.2617   2.821      3.12597
 3.0  27.0   9.70897    9.99132  0.521222   1.56398

In [63]:
wh1r = reshape(wh1, (:, 1, 2))

3×1×2 Array{Float32,3}:
[:, :, 1] =
  3.625
  4.875
 11.65625

[:, :, 2] =
  2.8125
  6.1875
 10.1875

In [70]:
wh12 = reshape(prod(wh1r, dims=3), (:, 1))

3×1 Array{Float32,2}:
  10.1953125
  30.164062
 118.74805

In [44]:
wh2

13×2 Array{Float64,2}:
 9.86233   12.4429
 8.34153    9.04732
 3.82099   12.7668
 4.01639   11.2871
 7.38176    7.40737
 0.841347   3.27922
 3.54392    0.611039
 1.94553    7.04704
 0.472056   0.97838
 1.83747    7.30306
 1.15599    6.0937
 2.821      3.12597
 0.521222   1.56398

In [73]:
wh2r = reshape(wh2, (1, :, 2))

1×13×2 Array{Float64,3}:
[:, :, 1] =
 9.86233  8.34153  3.82099  4.01639  …  1.83747  1.15599  2.821  0.521222

[:, :, 2] =
 12.4429  9.04732  12.7668  11.2871  …  7.30306  6.0937  3.12597  1.56398

In [74]:
wh22 = reshape(prod(wh2r, dims=3), (1, :))

1×13 Array{Float64,2}:
 122.716  75.4685  48.7817  45.3336  …  13.4192  7.04423  8.81835  0.81518

In [76]:
wh12 .+ wh22

3×13 Array{Float64,2}:
 132.911   85.6638   58.977    55.5289  …   17.2395   19.0137   11.0105
 152.88   105.633    78.9458   75.4976      37.2083   38.9824   30.9792
 241.464  194.217   167.53    164.082      125.792   127.566   119.563

In [59]:
reshape(prod(a, dims=3), (3, 13))

3×13 Array{Float64,2}:
  10.1953  10.1953  10.1953  10.1953  …   5.16789  3.25121  7.93406  0.81518
  30.1641  30.1641  23.6424  24.8514     11.3694   7.04423  8.81835  0.81518
 100.473   75.4685  38.9263  40.917      13.4192   7.04423  8.81835  0.81518

In [61]:
inter = build_targets(yolo_out, yy, darknet)


3×13 Array{Float64,2}:
  10.1953  10.1953  10.1953  10.1953  …   5.16789  3.25121  7.93406  0.81518
  30.1641  30.1641  23.6424  24.8514     11.3694   7.04423  8.81835  0.81518
 100.473   75.4685  38.9263  40.917      13.4192   7.04423  8.81835  0.81518

In [50]:
targets = nothing 

for (i, t) in enumerate(yy[2:end])
    println(i)

end
gg = hcat(yy[2]...)
# gg = vcat(ones(1, 2), gg)

1
2


5×2 Array{Float64,2}:
 60.0       78.0
  0.51093    0.858305
  0.442073   0.073521
  0.978141   0.074922
  0.872188   0.059833

13×6 Array{Float64,2}:
 1.0  21.0   7.01665    6.77858  9.86233   12.4429
 1.0  21.0   5.2451     6.11928  8.34153    9.04732
 1.0  21.0  11.0895     6.41263  3.82099   12.7668
 2.0   1.0   2.0082     5.64357  4.01639   11.2871
 2.0   1.0   8.8661     7.4662   7.38176    7.40737
 2.0  36.0   0.420667   2.62042  0.841347   3.27922
 2.0  35.0   1.77197   10.2312   3.54392    0.611039
 3.0  73.0   5.39256    9.47648  1.94553    7.04704
 3.0  75.0   9.54261    4.01509  0.472056   0.97838
 3.0   1.0   7.56092    9.29958  1.83747    7.30306
 3.0   1.0   9.50228    9.95315  1.15599    6.0937
 3.0  70.0   1.90308   11.2617   2.821      3.12597
 3.0  27.0   9.70897    9.99132  0.521222   1.56398

In [48]:
length(yy[2])


2

In [6]:
include("utils/utils.jl")



build_targets (generic function with 1 method)